In [1]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.json as pjson
import polars as pl
from tqdm.notebook import tqdm

import json
from datetime import date
from pathlib import Path

content_path = Path("/Users/davidenicoli/Local_Workspace/Datasets/ARPA/PIEMONTE/")
db_path = content_path / "site-data"
stazioni_path = content_path / "stazioni_meteo.json"


def dati_giornalieri_request_url(
    start_date="2000-1-1", stop_date="2023-6-30", id_punto_misura=""
):
    return f"https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo={id_punto_misura}&data_min={start_date}&data_max={stop_date}"

In [2]:
if not stazioni_path.exists():
    content = requests.get(
        "https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/"
    ).content
    stazioni = json.loads(content)
    with open(stazioni_path, "wb") as f:
        f.write(content)
else:
    with open(stazioni_path, "rt") as f:
        stazioni = json.load(f)

In [4]:
data = pl.DataFrame(
    stazioni["results"],
    schema_overrides={
        "url": pl.Utf8(),
        "codice_istat_comune": pl.Utf8(),
        "sensori_meteo": pl.List(
            pl.Struct(
                {
                    "url": pl.Utf8(),
                    "id_parametro": pl.Utf8(),
                    "data_inizio": pl.Utf8(),
                    "data_fine": pl.Utf8(),
                    "quota_da_pc": pl.Float32(),
                    "altezza_supporto": pl.Float32(),
                    "note": pl.Utf8(),
                    "fk_id_stazione_meteorologica": pl.Utf8(),
                }
            )
        ),
    },
)

In [18]:
thermo_sensors = (
    data.rename({"url": "url_stazione"})
    .explode("sensori_meteo")
    .with_columns(
        pl.col("sensori_meteo").struct.rename_fields(
            [
                "url_sensore",
                "id_parametro",
                "data_inizio_sensore",
                "data_fine_sensore",
                "quota_da_pc",
                "altezza_supporto",
                "note_sensore",
                "copy_url_stazione",
            ]
        )
    )
    .unnest("sensori_meteo")
    .filter(pl.col("id_parametro") == "TERMA")
)
thermo_sensors.write_csv(content_path / "thermo_sensors.csv", quote_style="always")

In [8]:
thermo_sensors = pl.read_csv(content_path / "thermo_sensors.csv")
urls = thermo_sensors.get_column("fk_id_punto_misura_meteo").to_list()
with requests.Session() as session:
    station_destinations = list(map(lambda url: session.get(url).json(), tqdm(urls)))
punto_misura_meteo = pl.from_dicts(
    station_destinations,
    schema_overrides={"codice_istat_comune": pl.Utf8(), "progr_punto_com": pl.Utf8()},
).explode("stazioni_meteorologiche")
punto_misura_meteo.write_csv(content_path / "punto_misura_meteo.csv")

  0%|          | 0/324 [00:00<?, ?it/s]

In [7]:
punto_misura_meteo = (
    pl.read_csv(
        content_path / "punto_misura_meteo.csv",
        dtypes={"codice_istat_comune": pl.Utf8(), "progr_punto_com": pl.Utf8()},
    )
    .filter(pl.col("data_fine_dati").str.to_date(r"%Y-%m-%d") > date(2000, 1, 1))
    .with_columns(
        pl.concat_str(
            [pl.lit("PIE"), pl.col("codice_istat_comune"), pl.col("progr_punto_com")],
            separator="-",
        ).alias("id_punto_misura")
    )
)

In [17]:
punto_misura_meteo.filter(pl.col("id_punto_misura").is_duplicated())["dati_giornalieri_meteo"].to_list()

['https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001019-901',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001019-901',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001022-905',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001022-905',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001073-901',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001073-901',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001175-902',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001175-902',
 'https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_meteo/?fk_id_punto_misura_meteo=PIE-001179-900',
 

In [3]:
def dwn_data(
    id_punto,
    session: requests.Session,
    path,
    start_date="2000-1-1",
    stop_date="2023-6-30",
):
    page = session.get(
        dati_giornalieri_request_url(start_date, stop_date, id_punto)
    ).json()
    data = page["results"]
    n = page["count"]
    with tqdm(total=n // 366, position=1, leave=False) as bar:
        while page["next"] is not None:
            page = session.get(page["next"]).json()
            data.extend(page["results"])
            bar.update()
    pl.from_dicts(
        data,
        schema={
            "url": pl.Utf8(),
            "data": pl.Utf8(),
            "tmedia": pl.Float64(),
            "tmax": pl.Float64(),
            "tmin": pl.Float64(),
            "tclasse": pl.Utf8(),
            "ptot": pl.Float64(),
            "pclasse": pl.Utf8(),
            "vmedia": pl.Float64(),
            "vraffica": pl.Float64(),
            "settore_prevalente": pl.Utf8(),
            "tempo_permanenza": pl.Utf8(),
            "durata_calma": pl.Utf8(),
            "vclasse": pl.Utf8(),
            "umedia": pl.Float64(),
            "umin": pl.Float64(),
            "umax": pl.Float64(),
            "uclasse": pl.Utf8(),
            "rtot": pl.Utf8(),
            "rclasse": pl.Utf8(),
            "fk_id_punto_misura_meteo": pl.Utf8(),
        },
    ).with_columns(
        pl.col("data").str.to_date(r"%Y-%m-%d"), pl.lit(id_punto).alias("id_punto")
    ).write_parquet(
        path
    )

In [ ]:
from time import sleep

ids = punto_misura_meteo["id_punto_misura"].to_list()
with requests.Session() as session:
    for id in tqdm(ids):
        path = db_path / f"{id}.parquet"
        if not path.exists():
            dwn_data(id, session, path, "2000-1-1", "2023-6-30")
            sleep(3)

In [18]:
pl.read_csv(
    content_path / "thermo_sensors.csv", dtypes={"codice_istat_comune": pl.Utf8()}
).with_columns(
    pl.col("fk_id_punto_misura_meteo")
    .str.split("/")
    .list.get(-2)
    .alias("id_punto_misura")
).write_csv(content_path / "metadata.csv")

In [19]:
thermo_sensors

url_stazione,url_sensore,id_parametro,data_inizio_sensore,data_fine_sensore,quota_da_pc,altezza_supporto,note_sensore,copy_url_stazione,codice_istat_comune,codice_stazione,denominazione,indirizzo_localita,nazione,longitudine_e_wgs84_d,latitudine_n_wgs84_d,quota_stazione,esposizione,note,tipo_staz,data_inizio,data_fine,sigla_prov,comune,fk_id_punto_misura_meteo
str,str,str,str,str,f64,f64,str,str,i64,str,str,str,str,f64,f64,i64,str,str,str,str,str,str,str,str
"""https://utilit…","""https://utilit…","""TERMA""","""1993-07-22""","""""",0.0,3.08,"""""","""https://utilit…",1003,"""250""","""ALA DI STURA""","""VIVAIO FORESTA…","""ITALIA""",7.31028,45.3125,1006,"""N""","""""","""HPT""","""1993-07-22""","""""","""TO""","""ALA DI STURA""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1999-09-08""","""""",0.0,3.78,"""""","""https://utilit…",1010,"""346""","""ANDRATE PINALB…","""PINALBA""","""ITALIA""",7.89222,45.56111,1580,"""SSE""","""""","""PRT""","""1999-09-09""","""""","""TO""","""ANDRATE""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1999-08-05""","""""",0.0,3.58,"""""","""https://utilit…",1011,"""353""","""VACCERA""","""VACCERA""","""ITALIA""",7.20167,44.875,1435,"""S""","""""","""PT""","""1999-08-06""","""""","""TO""","""ANGROGNA""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1991-01-09""","""""",0.0,1.68,"""""","""https://utilit…",1013,"""142""","""AVIGLIANA""","""GRANGIA""","""ITALIA""",7.39472,45.09389,340,"""N""","""""","""HPRTV""","""1991-01-09""","""""","""TO""","""AVIGLIANA""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1987-09-25""","""""",0.0,6.0,"""""","""https://utilit…",1019,"""004""","""RIFUGIO GASTAL…","""RIFUGIO GASTAL…","""ITALIA""",7.14333,45.29806,2659,"""N""","""""","""NPRTV""","""1987-09-26""","""""","""TO""","""BALME""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""2006-09-28""","""2018-04-17""",0.0,0.0,"""""","""https://utilit…",1019,"""S4101""","""BALME""","""FRAZIONE CORNE…","""ITALIA""",7.22194,45.3025,1410,"""ENE""","""""","""PTV""","""2006-09-29""","""""","""TO""","""BALME""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""2018-04-18""","""""",0.0,3.0,"""""","""https://utilit…",1019,"""S4101""","""BALME""","""FRAZIONE CORNE…","""ITALIA""",7.22194,45.3025,1410,"""ENE""","""""","""PTV""","""2006-09-29""","""""","""TO""","""BALME""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1990-12-06""","""""",0.0,9.25,"""""","""https://utilit…",1022,"""152""","""PRERICHARD""","""PRERICHARD""","""ITALIA""",6.71639,45.075,1290,"""WSW""","""""","""BHNPRTV""","""1990-12-06""","""""","""TO""","""BARDONECCHIA""","""https://utilit…"
"""https://utilit…","""https://utilit…","""TERMA""","""1990-11-09""","""2005-08-30""",0.0,9.5,"""""","""https://utilit…",1022,"""011""","""CAMINI FREJUS""","""CAMINI FREJUS""","""ITALIA""",6.68861,45.11722,1740,"""WSW""","""""","""BHNPRTV""","""1990-11-09""","""2005-08-30""","""TO""","""BARDONECCHIA""","""https://utilit…"
